# Detección de sombras 

*: El "ejemplo nubes" sale de acá: https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless

## Autenticación

In [26]:
import os
import ee
from google.auth.transport.requests import AuthorizedSession


##################################################################
# CREDENTIALS =======
print('\nEnviando credenciales a la nube ...')

PROJECT = 'pruebas-gee-00' # Ej: pruebas-engine-00
# SERVICE_ACCOUNT = 'pruebas-gee-jmb@pruebas-gee-00.iam.gserviceaccount.com'
SERVICE_ACCOUNT = 'jumanbar@gmail.com'
KEY = os.path.join(os.getcwd(), 'debian-key.json')
rest_api_url = 'https://earthengine.googleapis.com/v1beta/projects/{}/table:computeFeatures'

print('\LLAVE:', KEY)

# PROBLEMAS ACA / INICIO
credentials = ee.ServiceAccountCredentials(SERVICE_ACCOUNT, KEY)
scoped_credentials = credentials.with_scopes(
    ['https://www.googleapis.com/auth/cloud-platform'])

session = AuthorizedSession(scoped_credentials)
ee.Initialize(credentials)
# PROBLEMAS ACA / FIN
###################################################################


Enviando credenciales a la nube ...
\LLAVE: c:\Users\juan.barreneche\Documents\gee_automatizacion\debian-key.json


In [27]:

area_nubes_rn = ee.Geometry.Polygon([[[-58.06659211642004,-33.66519274203066],
                                      [-55.16620149142004,-33.66519274203066],
                                      [-55.16620149142004,-31.96641660082516],
                                      [-58.06659211642004,-31.96641660082516],
                                      [-58.06659211642004,-33.66519274203066]]])
print(area_nubes_rn.centroid().getInfo())

{'type': 'Point', 'coordinates': [-56.61639680391994, -32.82145465824571]}


## Funciones


### Variables y tal

In [39]:
"""
# VARIABLES, VALORES INICIALES
# Definir estas cosas para que el VS code no ponga
# alertas todo el tiempo
id_zona = 1
zona = 'PALMAR'
geom = ee.FeatureCollection('users/brunogda/zonas_palmar_represa_dis')\
    .first()\
    .geometry()

p = [10, 50, 90]
cloud_perc = 25
cloud_perc2 = 25
MAX_CLOUD_PROBABILITY = 10

ini_date = '2023-03-01'
end_date = '2023-03-31'
mask_ndwi = ee.Image(0)
lo_ndwi = ee.Image(0)

# Para sombras
BUFFER = 50
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.5
CLD_PRJ_DIST = 1
"""

from math import tan, pi, log10, ceil
from pprint import pprint
import numpy as np
from inifun import *
import calfun as cf
id_zona = 5
nzonas = 60
isValidZoneID(id_zona, nzonas)

def getAssetString(nzonas, id_zona):    
    if nzonas == 7:
        out = getAssetFromIdZona(id_zona)
    else:
        out = getAssetFromIdZona60(id_zona)
    return out

asset_string = getAssetString(nzonas, id_zona)

print(f'\tAsset string: {asset_string}')

# Polígono:
geom = ee.FeatureCollection(asset_string).first().geometry()

p = [10, 50, 90]
cloud_perc  = 50
cloud_perc2 = 50
# MAX_CLOUD_PROBABILITY = 10
MAX_CLOUD_PROBABILITY = 40

ini_date = '2023-06-17'
ini_date_o3 = '2023-05-17'
end_date = '2023-06-18'

escala_proy_nubes = 50

# Cloud shadows
BUFFER = 50
CLD_PRB_THRESH = 50
CLD_PRJ_DIST = 1

# COPIAR VARIABLES AL NAMESPACE DE calfun:
cf.zona = 'RN60Z'
cf.id_zona = id_zona
cf.geom = geom
cf.p = p
cf.cloud_perc = cloud_perc
cf.cloud_perc2 = cloud_perc2
cf.MAX_CLOUD_PROBABILITY = MAX_CLOUD_PROBABILITY
cf.end_date = end_date
cf.ini_date = ini_date
cf.ini_date_o3 = ini_date_o3

def filterProperties(nzonas, id_zona):  
    # DEFINIR PROPIEDAD Y VALOR PARA FILTRAR S2:
    prp = 'MGRS_TILE'
    val = '21HVD'

    if nzonas == 7:
        if id_zona >= 4 and id_zona < 6:
            prp = 'SENSING_ORBIT_NUMBER'
            val = 124
        if id_zona >= 6:
            val = '21HWD'
    else:
        if id_zona >= 1 and id_zona < 18:
            val = '21HVD'

        if id_zona == 18:
            val = '21HWD'

        if id_zona >= 19 and id_zona < 37:
            prp = 'SENSING_ORBIT_NUMBER'
            val = 124

        if id_zona >= 37 and id_zona < 56:
            val = '21HWD'

        if id_zona >= 56 and id_zona < 59:
            val = '21HXD'

        if id_zona >= 59 and id_zona <= 60:
            val = '21HXE'
    return { 'prp': prp, 'val': val }

fprop = filterProperties(60, 5)
print(f'\tid_zona:  {id_zona}')
print(f'\tnzonas:   {nzonas}')
print(f'\tProperty: {fprop["prp"]}')
print(f'\tValue:    {fprop["val"]}')

	Asset string: users/brunogda/RN60/05_rn
	id_zona:  5
	nzonas:   60
	Property: MGRS_TILE
	Value:    21HVD


### S5Clouds

In [29]:
from datetime import timedelta, datetime, timezone

def dtRange(dt, hour_diff=6):
    dt_0 = str(dt + timedelta(hours=-hour_diff)).replace(' ', 'T')
    dt_1 = str(dt + timedelta(hours=hour_diff)).replace(' ', 'T')
    return([dt_0, dt_1])


def cloudsBaseHeightByDate(date):
    cbh = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CLOUD')\
        .filterDate(date.advance(-4, 'hour'), date.advance(4, 'hour'))\
        .map(lambda img: img.clip(area_nubes_rn))\
        .select('cloud_base_height')

    p70 = cbh.mean().log10().reduceRegion(
        reducer = ee.Reducer.percentile([70]),
        geometry = area_nubes_rn,
        scale = 1113.2
    ).get('cloud_base_height')
    
    return ee.Number(10).pow(ee.Number(p70))


def shadowDistFromClouds(img):
    dt = ee.Date(img.date())
    # print(f'Fecha: {str(dt)}')
    # zenit = img.getInfo()['properties']['MEAN_SOLAR_ZENITH_ANGLE']
    # zenit = img.get('MEAN_SOLAR_ZENITH_ANGLE')
    zenit = ee.Number(img.get('MEAN_SOLAR_ZENITH_ANGLE'))
    # altitude_ang = 90 - zenit
    altitude_ang = ee.Number(90).subtract(zenit)
    # print(f'Mean solar zenith angle:\t{zenit:.2f}')
    # print(f'Mean solar altitude angle:\t{altitude_ang:.2f}')
    clouds_height = cloudsBaseHeightByDate(dt)
    #  print(f'Altura de nubes (percentil 70): {clouds_height:.2f}')
    
    # d_nubes = clouds_height / tan(pi * altitude_ang / 180)
    d_nubes = clouds_height.divide(
        ee.Number(pi).multiply(
            altitude_ang.divide(ee.Number(180))
            ).tan()
        )
    return d_nubes

## Distancia sombra - nube (demo)

In [30]:
MSI_source = 'COPERNICUS/S2_HARMONIZED'
MSI = ee.ImageCollection(MSI_source)
print(f'ini: {ini_date} - fin: {end_date}')
criteria = ee.Filter.And(ee.Filter.bounds(geom),
                         ee.Filter.date(ini_date, end_date))

x = MSI.filter(criteria).filter(ee.Filter.eq(fprop['prp'], fprop['val']))
a = x.first().date().getInfo()['value']
print(f'Fecha de primer imagen de la colección S2:\n\t{datetime.fromtimestamp(a/1000)}')
a2 = x.first().get('GENERATION_TIME').getInfo()

nubes = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CLOUD')\
        .filterDate(x.first().date().advance(-4, 'hour'), x.first().date().advance(4, 'hour'))

clouds_height = cloudsBaseHeightByDate(x.first().date())
print(f'Altura de nubes (percentil 70): {clouds_height.getInfo():.2f}m')

d = shadowDistFromClouds(x.first())

# print(datetime.fromtimestamp(x.first().date().getInfo()))
print(f'Distancia en el plano de la sombra a la nube: {d.getInfo():.2f}m => {ceil(d.getInfo()/escala_proy_nubes)} px')

ini: 2023-06-17 - fin: 2023-06-18
Fecha de primer imagen de la colección S2:
	2023-06-17 11:01:46.681000
Altura de nubes (percentil 70): 461.85m
Distancia en el plano de la sombra a la nube: 872.90m => 18 px


In [31]:

print(datetime.fromtimestamp(1686997020000/1000))
print(datetime.fromtimestamp(1655901773000/1000))
print(datetime.fromtimestamp(1687010506681/1000))
print(datetime.fromtimestamp(1687010506681/1000, timezone.utc))

2023-06-17 07:17:00
2022-06-22 09:42:53
2023-06-17 11:01:46.681000
2023-06-17 14:01:46.681000+00:00


### Funciones nuevas

In [32]:
###################################################################
# PARA GESTIONAR SOMBRAS DE NUBES
# Fuente:
# https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless


def add_cloud_bands(img):
    # Basada en maskClouds (ver Actuales)
    cld_prb = ee.Image(img.get('cloud_mask')).select('probability')
    resolution_reducer = ee.Reducer.anyNonZero()
    is_cloud = (
        cld_prb
        .gte(MAX_CLOUD_PROBABILITY)
        .rename('clouds')
        .select('clouds')
        .setDefaultProjection(img.select(0).projection())
        .reduceResolution(resolution_reducer, True, (escala_proy_nubes / 10) ** 2)
        .reproject(**{'crs': img.select(0).projection(), 'scale': escala_proy_nubes})
    )
    return img.addBands(ee.Image([cld_prb, is_cloud]))


def add_shadow_bands(img):
    # Distancia, en el plano, estimada de las nubes hasta su sombra, basada en el ángulo del sol:
    d_nubes = (
        shadowDistFromClouds(img)
        .divide(ee.Number(escala_proy_nubes))
        .ceil()
    )
    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));
    cld_proj = (
        img
        .select('clouds')
        .directionalDistanceTransform(shadow_azimuth, d_nubes)
        .reproject(**{'crs': img.select(0).projection(), 'scale': escala_proy_nubes})
        .select('distance')
        .rename('cloud_transform')
    )
    return img.addBands(ee.Image(cld_proj))


def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)
    # img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_or_shdwproj = (
        img_cloud_shadow
        .select('cloud_transform')
        .add(img_shadow.select('clouds'))
        .gt(0)
        .rename('cloudmask')
    )

    # # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    # is_cld_shdw = (
    #     is_cld_shdw
    #     .focalMin(2)
    #     .focalMax(BUFFER * 2 / 20)
    #     .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
    #     .rename('cloudmask')
    #     )

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.updateMask(is_cld_or_shdwproj)

###################################################################

## Image Collections

In [33]:
MSI_source = 'COPERNICUS/S2_HARMONIZED'    # Nuestro, original
# MSI_source = 'COPERNICUS/S2_SR'            # Ejemplo nubes*
# MSI_source = 'COPERNICUS/S2_SR_HARMONIZED' # Otra prueba...

# Sentinel multibanda
MSI = ee.ImageCollection(MSI_source)

# Probabilidad de nubes:
S2_clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')

print(f'ini: {ini_date} - fin: {end_date}')
# Filter input collections by desired data range and region.
criteria = ee.Filter.And(ee.Filter.bounds(geom),
                         ee.Filter.date(ini_date, end_date))

S2_clouds = S2_clouds.filter(criteria)

# Esto se fija en qué imágenes de la colección hay pocas nubes en total y se
# queda con esas:
S2_mask = MSI.filter(criteria)\
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', "less_than", cloud_perc2)

# FILTER Sentinel 2 collection

FC2 = S2_mask.filter(ee.Filter.eq(fprop["prp"], fprop["val"])).map(cf.maskEdges)
d_nubes = shadowDistFromClouds(FC2.first())
pprint(f'Dist. nubes en el plano: {d_nubes.getInfo()}')

## NDWI
## Sirve con S2_HARMONIZED y S2_SR_HARMONIZED
ndwi = S2_mask.median().normalizedDifference(['B3', 'B8'])
hi_ndwi = ndwi.select('nd').gte(0.2)
lo_ndwi = ndwi.select('nd').lt(0.2)
mask_ndwi = hi_ndwi.updateMask(hi_ndwi)

# FILTER Sentinel 2 collection
# FC2 = MSI.filterDate(ini_date, end_date)\
#     .filterBounds(geom)\
#     .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', "less_than", cloud_perc)\
#     .filter(ee.Filter.eq(prp, val))

# Join S2 SR with cloud probability dataset to add cloud mask.
FC2_with_cloud_mask = ee.ImageCollection(
    ee.Join.saveFirst('cloud_mask').apply(
        primary = FC2,
        secondary = S2_clouds,
        condition = ee.Filter.equals(
            leftField = 'system:index',
            rightField = 'system:index')
        )
    )

S2_cloud_masked = FC2_with_cloud_mask.map(add_cld_shdw_mask)
# pprint(S2_cloud_masked.first().getInfo())
# cf.printBandNames(S2_cloud_masked.first())

img_cloud = add_cloud_bands(FC2_with_cloud_mask.first())
# pprint(img.getInfo())


# # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
# shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

# resolution_reducer = ee.Reducer.anyNonZero()
# # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.

# cf.printBandNames(img)
# img_cloud_red = (
#     img
#     .select('clouds')
#     .setDefaultProjection(img.select(0).projection())
#     .reduceResolution(resolution_reducer, True, (escala_proy_nubes / 10) ** 2)
#     .reproject(**{'crs': img.select(0).projection(), 'scale': escala_proy_nubes})
# )
# cf.printBandNames(img_cloud_red)

# cld_proj = (
#     img_cloud_red
#     .select('clouds')
#     .directionalDistanceTransform(shadow_azimuth, d_nubes.divide(ee.Number(escala_proy_nubes)).ceil())
#     # .directionalDistanceTransform(shadow_azimuth.multiply(ee.Number(-1)), ceil(d_nubes/escala_proy_nubes))
#     .reproject(**{'crs': img.select(0).projection(), 'scale': escala_proy_nubes})
#     .select('distance')
#     # .mask()
#     .rename('cloud_transform')
# )

img_shadow = add_shadow_bands(img_cloud)
cf.printBandNames(img_shadow)
is_cld_or_shdwproj = img_shadow.select('cloud_transform').add(img_shadow.select('clouds')).gt(0)
# is_cld_shdw = img_shadow.select('cloud_transform').gt(0)
img_shadow = img_shadow.updateMask(is_cld_or_shdwproj)
# img.addBands(cld_proj)
# d_nubes.divide(ee.Number(escala_proy_nubes)).ceil()
cf.printBandNames(img_shadow)

ini: 2023-06-17 - fin: 2023-06-18
'Dist. nubes en el plano: 872.9035036265947'
B1, B2, B3, B4, B5, B6, B7, B8, B8A, B9, B10, B11, B12, QA10, QA20, QA60, MSK_CLASSI_OPAQUE
MSK_CLASSI_CIRRUS, MSK_CLASSI_SNOW_ICE, probability, clouds, cloud_transform
B1, B2, B3, B4, B5, B6, B7, B8, B8A, B9, B10, B11, B12, QA10, QA20, QA60, MSK_CLASSI_OPAQUE
MSK_CLASSI_CIRRUS, MSK_CLASSI_SNOW_ICE, probability, clouds, cloud_transform


## Run



### Proyección nubes

Esta parte es extraída de una de las funciones (ver [Nuevas](#scrollTo=Nuevas)). Lo puse acá porque quería ver cómo funcionaba eso de la proyección y la función `directionalDistanceTransform`

In [34]:
import geemap
m = geemap.Map()
d = shadowDistFromClouds(x.first()).getInfo()
test = S2_cloud_masked.first()
cf.printBandNames(test)
maxdist = ceil(d/escala_proy_nubes)
m.add_layer(test, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1}, 'test', True, 1)
m.add_layer(FC2.first(), {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1}, 'FC2', True, 1)
m.add_layer(img_shadow.select('cloud_transform'), {'min': 0, 'max': maxdist, 'palette': ['yellow', 'red']}, 'DISTANCE2', True, 1)
m.add_layer(img_shadow.mask(), None, 'Clouds mask', True, 1)
m.add_layer(test.mask(), None, 'test Clouds mask', True, 1)
m.set_zoom(12)
m.set_center(lat=-33.09039099883797, lon=-57.306747436523445)
display(m)

B1, B2, B3, B4, B5, B6, B7, B8, B8A, B9, B10, B11, B12, QA10, QA20, QA60, MSK_CLASSI_OPAQUE
MSK_CLASSI_CIRRUS, MSK_CLASSI_SNOW_ICE, probability, clouds, cloud_transform


Map(center=[-33.09039099883797, -57.306747436523445], controls=(WidgetControl(options=['position', 'transparen…

## Viz

### Para ver las bandas del FC2_with_cloud_mask:

In [ ]:
# pprint.pprint(FC2_with_cloud_mask.getInfo()['features'][0]['bands'])
# ! C:/Users/juan.barreneche/AppData/Local/Programs/Python/Programs/Python/Python310/python.exe -m pip install geemap

Posible estimación de alturas de nubes: https://code.earthengine.google.com/448f9425387963431ec1ac567b206bc5; (https://code.earthengine.google.com/?scriptPath=users%2Fjumanbar%2FpruebasRN%3AAlturaNubes)

### Armar el mapa

Acá simplemente es agregar y probar capas para visualizar. Atento a la función `.selfMask` que ayudó a que se vea lo deseado.

In [ ]:
# m.centerObject(geom, 12)
# -33.09039099883797, -57.306747436523445
m.set_center(lat=-33.09039099883797, lon=-57.306747436523445)
estilo_zona = {'fillColor': '#ffcc9988', 'width': 0}
# estilo_buffer = {'fillColor': '#66003388', 'width': 0}
zona_feature = ee.FeatureCollection(geom)
# buffer_feature = ee.FeatureCollection(geom_buffer)
# m.addLayer(buffer_feature.style(**estilo_buffer), {}, 'Buffer')
m.addLayer(zona_feature.style(**estilo_zona), {}, 'Área de medición de nubes', False)
display(m)